# **MY Experiments**

# Import Packages


Этот проект будет структурирован следующим образом:

```
Search_by_summary/
├── experiments/
│   ├── Summary_experiments.ipynb
│   └── Summary_experiments2.ipynb
├── my_module/
│   ├── data_setup.py
│   ├── engine.py
│   ├── model.py
│   ├── train.py
│   └── utils.py
├── tensors/
│   ├── summary_tensors1.pkl
│   └── summary_tensors.pkl
└── data/
    └── wine_reviews.csv/

```

Здесь каталог верхнего уровня называется `first_modulet`.

Внутри есть еще один каталог `my_module`, который содержит несколько файлов `.py`, назначение которых может быть следующим:
* `data_setup.py` - файл для подготовки данных (и загрузки данных при необходимости).
* `engine.py` - файл, содержащий различные функции обучения модели.
* `model_builder.py` or `model.py` - файл для создания модели PyTorch.
* `train.py` - файл для использования всех остальных файлов и обучения целевой модели PyTorch.
* `utils.py` -файл, посвященный полезным утилитам.

А каталоги `models` и `data` могут содержать модели и файлы данных PyTorch соответственно

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import pickle
import torch
import matplotlib.pyplot as plt
from time import time
# import nltk
import pickle
import torch.cuda as cuda

In [2]:
# %%capture
# !pip install transformers
from transformers import AutoTokenizer
from transformers import pipeline

# Import Dataset


In [3]:
import os
from pathlib import Path

In [4]:
# Setup path to data folder
data_path = Path("../data/")
# Setup train and testing paths
train_dir = data_path / "train"
test_dir = data_path / "test"
train_dir, test_dir


(WindowsPath('../data/train'), WindowsPath('../data/test'))

In [10]:
dataset = pd.read_csv('../data/train/dataset100.csv', sep='\t')

In [5]:
dataset1 = pd.read_csv("../data/train/wine_reviews.csv", usecols =['title', 'description', 'points','variety','price'], encoding='latin1')
dataset1["description"] = dataset1["description"].astype(pd.StringDtype())
dataset1 = dataset1.dropna().reset_index()
dataset1.rename(columns = {'index':'id'}, inplace = True )

In [5]:
%%writefile ../my_module/data_setup.py
"""
Contains functionality for loading particular columns of data, changing types of needed columns
and creating na ID column
"""
import os
from pathlib import Path

# Setup path to data folder
data_path = Path("../data/")
# Setup train and testing paths
train_dir = data_path / "train"
test_dir = data_path / "test"

# dataset = pd.read_csv('../data/train/dataset100.csv', sep='\t')

dataset = pd.read_csv("../data/train/wine_reviews.csv", usecols =['title', 'description', 'points','variety','price'], encoding='latin1')
dataset["description"] = dataset["description"].astype(pd.StringDtype())
dataset = dataset.dropna().reset_index()
dataset.rename(columns = {'index':'id'}, inplace = True )





Overwriting ../my_module/data_setup.py


In [9]:

from transformers import AutoTokenizer
from transformers import pipeline
# Load the tokenizer for the specific model
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")
# Load the summarization pipeline
summarizer = pipeline("summarization", model='facebook/bart-large-cnn', min_length=15, max_length=35)

In [24]:
# %%writefile ../my_module/model_builder.py
"""
Contain summarizing pipeline that is loaded from pipeline() using the following task identifier: "summarization".
The models that this pipeline use is 'bart-large-cnn'.
See the up-to-date list of available models on huggingface.co/models.
"""
from transformers import AutoTokenizer
from transformers import pipeline


# Load the tokenizer for the specific model
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")
# Load the summarization pipeline
summarizer = pipeline("summarization", model='facebook/bart-large-cnn')
DESCR = dataset['description'][4]
summary = summarizer(DESCR, min_length = 15,max_length = 35)
print(DESCR)
print(summary[0]['summary_text'])


In [10]:
import os, sys
parentPath = os.path.abspath("../")
if parentPath not in sys.path:
    sys.path.insert(0, parentPath)

In [11]:
print(sys.path)

['e:\\Обучение в IT-academy\\Summary_module', 'e:\\Обучение в IT-academy\\Summary_module\\experiments', 'c:\\Users\\38\\anaconda3\\python39.zip', 'c:\\Users\\38\\anaconda3\\DLLs', 'c:\\Users\\38\\anaconda3\\lib', 'c:\\Users\\38\\anaconda3', '', 'C:\\Users\\38\\AppData\\Roaming\\Python\\Python39\\site-packages', 'C:\\Users\\38\\AppData\\Roaming\\Python\\Python39\\site-packages\\win32', 'C:\\Users\\38\\AppData\\Roaming\\Python\\Python39\\site-packages\\win32\\lib', 'C:\\Users\\38\\AppData\\Roaming\\Python\\Python39\\site-packages\\Pythonwin', 'c:\\Users\\38\\anaconda3\\lib\\site-packages', 'c:\\Users\\38\\anaconda3\\lib\\site-packages\\win32', 'c:\\Users\\38\\anaconda3\\lib\\site-packages\\win32\\lib', 'c:\\Users\\38\\anaconda3\\lib\\site-packages\\Pythonwin']


In [12]:
# from transformers import AutoTokenizer
# from transformers import pipeline
import my_module
from my_module import model_builder

device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = model_builder.get_bart_tokenizer()
summarizer = model_builder.get_bart_pipline()


# Instantiate an instance of the model from the "model_builder.py" script
DESCR = dataset1['description'][4]
summary = summarizer(DESCR, min_length = 15,max_length = 35)
print(DESCR)
print(summary[0]['summary_text'])

Blackberry and raspberry aromas show a typical Navarran whiff of green herbs and, in this case, horseradish. In the mouth, this is fairly full bodied, with tomatoey acidity. Spicy, herbal flavors complement dark plum fruit, while the finish is fresh but grabby.
Blackberry and raspberry aromas show a typical Navarran whiff of green herbs and, in this case, horseradish. Spicy, herbal flavors complement


In [13]:
DESCR = dataset1['description'][5]
summary = summarizer(DESCR, min_length = 15,max_length = 35)
print(DESCR)
print(summary[0]['summary_text'])

Here's a bright, informal red that opens with aromas of candied berry, white pepper and savory herb that carry over to the palate. It's balanced with fresh acidity and soft tannins.
Bright, informal red opens with aromas of candied berry, white pepper and savory herb. It's balanced with fresh acidity and soft tann


I want to calculate the similarity between sentences by calculating the cosine distance between tensors obtained from the predictions of a sentence summarization model. While the summarization pipeline from the Hugging Face transformers library returns a list of summarized sentences as token indices, I can convert these token indices into tensors and then compute the cosine distance between them.

In order to count and store the summary_tensors for all offers in my dataset to enable quick loading and comparison with the summary_tensors of one input offer, I will follow these steps:

1. Iterate through your dataset using the summarization pipeline.

2. Convert and store the summary_tensors: For each wine_id, convert the token indices of the summary into tensors and store them in a data structure like a list or dictionary. So I can use the offer ID or any other unique identifier as the key for indexing the summary_tensors.

3. Save the summary_tensors to a file: To enable quick loading and comparison, I can save the summary_tensors to a file in a serialized format. I can use libraries like pickle or torch.save() to save the summary_tensors data structure to a file.

First I will try to do this with fist 1000 rows in dataset

In [14]:
dataset = dataset1[:5].copy()

In [23]:
# tensors_path = Path("../tensors/")
# # Setup train and testing paths
# tensors_dir = tensors_path / "train"
# E:\Обучение в IT-academy\Summary_module\my_module\tensors\summary_tensors_old.pkl
# Setup path to tensors folder
tensors_dir = Path("../my_module/tensors/")
tensors_path = tensors_dir / "summary_tensors.pkl"

summaries_dir = Path("../my_module/summaries/")
summaries_path = summaries_dir / "summaries.csv"

tensors_path, summaries_path

(WindowsPath('../my_module/tensors/summary_tensors.pkl'),
 WindowsPath('../my_module/summaries/summaries.csv'))

In [21]:
dataset = dataset1[:5].copy()
device = torch.device("cuda" if cuda.is_available() else "cpu")  # Set the device to

def get_tensors():
    # Store the summary tensors
    summary_tensors = {}
    # Iterate over the dataset and retrieve summaries
    for index, row in dataset.iterrows():
        index_id = row['id']
        text = row['description']

        # Use the summarization pipeline to generate summaries
        summary = summarizer(text,min_length = 15,max_length = 35)[0]["summary_text"]

        dataset.at[index, 'summary'] = summary
        # Tokenize the summary
        tokens = tokenizer(summary, truncation=True, padding=True)

        # Convert the token IDs to a tensor
        summary_tensor = torch.tensor(tokens["input_ids"]).unsqueeze(0).to(device)


        # Store the summary tensor with the offer ID as the key
        summary_tensors[index_id] = summary_tensor

    # Save the summary tensors to a file
    # with open("summary_tensors.pkl", "wb") as f:
    with open(tensors_path, "wb") as f:    
        # pickle.dump(summary_tensors, f)
        torch.save(summary_tensors, f)

    # Load the summary tensors from the file
    with open(tensors_path, "rb") as f:
        # loaded_summary_tensors 
        if cuda.is_available():  # Check if CUDA is available before loading the tensors
            loaded_summary_tensors = torch.load(f, map_location=torch.device('cuda'))
        else:
            loaded_summary_tensors = torch.load(f, map_location=torch.device('cpu'))
            
    return len(loaded_summary_tensors)

C:\Users\38\AppData\Local\Temp\ipykernel_12612\2686807156.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'This is ripe and fruity, a wine that is smooth while still structured. Firm tannins are filled out with juicy red berry fruits and freshen' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  dataset.at[index, 'summary'] = summary


In [24]:
dataset.to_csv(summaries_path, sep='\t')

In [22]:
dataset.head()

,id,description,points,price,title,variety,summary
0,1,"This is ripe and fruity, a wine that is smooth...",87,15.0,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,"This is ripe and fruity, a wine that is smooth..."
1,2,"Tart and snappy, the flavors of lime flesh and...",87,14.0,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,"Tart and snappy, the flavors of lime flesh and..."
2,3,"Pineapple rind, lemon pith and orange blossom ...",87,13.0,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,"Pineapple rind, lemon pith and orange blossom ..."
3,4,"Much like the regular bottling from 2012, this...",87,65.0,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,"This comes across as rather rough and tannic, ..."
4,5,Blackberry and raspberry aromas show a typical...,87,15.0,Tandem 2011 Ars In Vitro Tempranillo-Merlot (N...,Tempranillo-Merlot,Blackberry and raspberry aromas show a typical...


Lets try to get a summary to one example of description, tokenize the summary,
convert the token IDs to a tensor and calculate cosine similarity between input summary tensor and dataset summary tensors.

In [ ]:
text = dataset['description'][6]
# imput_text = normalize_text(imput_text)

summary = summarizer(text)[0]["summary_text"]

# Tokenize the summary
tokens = tokenizer(summary, truncation=True, padding=True)


    # Convert the token IDs to a tensor
input_summary_tensor = torch.tensor(tokens["input_ids"]).unsqueeze(0).to(device)

print(text)
print(summary)

This dry and restrained wine offers spice in profusion. Balanced with acidity and a firm texture, it's very much for food.
This dry and restrained wine offers spice in profusion. Balanced with acidity and a firm texture, it's very much for food.


In [ ]:
# Первый вариант, когда я считала на GPU
device = torch.device("cuda" if cuda.is_available() else "cpu")  # Set the device to
with open("summary_tensors.pkl", "rb") as f:
    loaded_summary_tensors = pickle.load(f)

loaded_summary_tensors = {k: v.to(device) for k, v in loaded_summary_tensors.items()}

In [ ]:
# Второй вариант, когда я загружаю файл на CPU
import torch.cuda as cuda
# import pickle

# Check if CUDA is available
device = torch.device("cuda" if cuda.is_available() else "cpu")

# Specify the complete file path to load the pkl file
file_path = '/content/summary_tensors.pkl'

# Load the summary tensors from the file
with open(file_path, "rb") as f:
    if cuda.is_available():  # Check if CUDA is available before loading the tensors
        loaded_summary_tensors = torch.load(f, map_location=torch.device('cuda'))
    else:
        loaded_summary_tensors = torch.load(f, map_location=torch.device('cpu'))

In [ ]:
input_summary_tensor.shape, summary_tensor.shape

(torch.Size([1, 29]), torch.Size([1, 32]))

In [ ]:
print(input_summary_tensor.device)
print(summary_tensor.device)

cpu
cpu


In [ ]:
from torch.nn.functional import cosine_similarity
loaded_summary_tensors = loaded_summary_tensors  # Dictionary of summary tensors loaded from file
# Calculate cosine similarity between input summary tensor and dataset summary tensors
similarity_scores = {}
for offer_id, summary_tensor in loaded_summary_tensors.items():
        # Adjust the size of tensors to match along the second dimension
    if input_summary_tensor.shape[1] > summary_tensor.shape[1]:
        summary_tensor = torch.nn.functional.pad(summary_tensor, (0, input_summary_tensor.shape[1] - summary_tensor.shape[1]))
    elif input_summary_tensor.shape[1] < summary_tensor.shape[1]:
        input_summary_tensor = torch.nn.functional.pad(input_summary_tensor, (0, summary_tensor.shape[1] - input_summary_tensor.shape[1]))
    # similarity = cosine_similarity(input_summary_tensor.unsqueeze(0).float(), summary_tensor.unsqueeze(0))
    similarity = cosine_similarity(input_summary_tensor.float(), summary_tensor)
    similarity_scores[offer_id] = similarity.item()

# Sort the similarity scores and get the top 5 most similar sentences
top_5_similar_sentences = sorted(similarity_scores.items(), key=lambda x: x[1], reverse=True)[:5]



# print(top_5_similar_sentences)
# Print the top 5 most similar sentences along with their similarity scores
for offer_id, similarity in top_5_similar_sentences:
    similarity = round(similarity, 4)
    sentence_text = dataset["summary"][offer_id]
    points = dataset["points"][offer_id]
    variety = dataset["variety"][offer_id]
    print(f"ID: {offer_id} | Points: {points} | Variety: {variety} |Similarity: {similarity} | Sentence: {sentence_text}")

ID: 7 | Points: 87 | Variety: GewÃ¼rztraminer |Similarity: 1.0 | Sentence: Savory dried thyme notes accent sunnier flavors of preserved peach. It's fruity and fresh, with an elegant, sprightly footprint.
ID: 9 | Points: 87 | Variety: Cabernet Sauvignon |Similarity: 0.585 | Sentence: Soft, supple plum envelopes an oaky structure in this Cabernet. Coffee and chocolate complete the picture, finishing strong at the end.
ID: 8 | Points: 87 | Variety: Pinot Gris |Similarity: 0.3241 | Sentence: This has great depth of flavor with its fresh apple and pear fruits. It's off dry while balanced with acidity and a crisp texture.
ID: 6 | Points: 87 | Variety: GewÃ¼rztraminer |Similarity: 0.3134 | Sentence: This dry and restrained wine offers spice in profusion. Balanced with acidity and a firm texture, it's very much for food.
ID: 2 | Points: 87 | Variety: Riesling |Similarity: 0.3095 | Sentence: Pineapple rind, lemon pith and orange blossom start off the aromas. The palate is a bit more opulent, wi